In [22]:
import os
import datetime
import csv
import json
import time
import re
import string
import requests
import selenium
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import   By 
from selenium.webdriver.firefox.options  import   Options 
from selenium.webdriver.support import   expected_conditions  as   EC 
from selenium.webdriver.support.wait  import   WebDriverWait 

In [23]:
cities=["L\'Aquila"
        ,'Teramo','Pescara','Chieti','Potenza','Matera','Cosenza','Catanzaro','Cosenza','Crotone','Vibo Valentia',
        'Reggio Calabria','Caserta','Benevento','Avellino','Salerno','Napoli','Piacenza','Parma','Reggio Emilia','Modena',
        'Ferrara','Ravenna','Rimini','Forli-Cesena Area','Bologna','Udine','Gorizia','Trieste','Pordenone','Viterbo','Rieti',
        'Latina','Frosinone','Roma','Imperia','Savona','La Spezia','Genova','Varese','Como','Sondrio','Bergamo','Brescia','Pavia',
        'Cremona','Mantova','Lecco','Lodi','Milano','Pesaro e Urbino','Ancona','Macerata','Ascoli Piceno',
        'Fermo','Campobasso','Isernia','Vercelli','Novara','Cuneo','Asti','Alessandria','Biella','Verbano-Cusio-Ossola','Torino',
        'Foggia','Taranto','Brindisi','Lecce','Barletta-Andria-Trani','Bari','Sassari','Nuoro','Oristano','Sardegna del Sud','Cagliari',
        'Trapani','Agrigento','Caltanissetta','Enna','Ragusa','Siracusa','Palermo','Messina','Catania','Lucca',
        'Pistoia','Livorno','Pisa','Arezzo','Siena','Grosseto','Prato','Firenze', 'Massa Carrara Area',
        'Bolzano','Trento','Perugia','Terni','Aosta','Verona','Vicenza','Belluno','Treviso','Padova',
        'Rovigo','Venezia'
       ]

In [24]:
country='Italia'
people=2
in_day=18  #si dovrebbe automatizzare la modifica di questa variabile nel caso in cui il processo sia svolto da remoto
in_month=6
in_year=2020
out_day=19
out_month=6
out_year=2020

today = datetime.date.today()
stringaGiorno=today.day
stringaMese=today.month
#path=""+str(today.day)+"-"+str(today.month)  #nome cartella del giorno attuale
dataOdierna=""+str(today.day)+"/"+str(today.month)+"/"+str(today.year) 

#driver.get(url)
    
def preparaDriver(link):
    '''Returns a Firefox Webdriver.'''
    options = Options()
    options.add_argument('-headless')  #modalita senza interfaccia grafica
    driver = Firefox() #invoco il webdriver di firefox geckodriver in path
    driver.get(link)
    wait = WebDriverWait(driver, timeout=10).until( #attesa implicita
        EC.presence_of_all_elements_located(
            (By.CLASS_NAME, 'sr-hotel__title')))
    return driver

def scrape_risultati(driver, n_results):      #restituisce la lista degli hotel
    time.sleep(4)
    clickCookie=driver.find_element_by_xpath('//*[@id="cookie_warning"]/div/div/div[2]/button/span').click()
    time.sleep(4) #richiede qualche secondo di riposo altrimenti non carica e va in errore
    clickDisponibilita=driver.find_element_by_xpath('//*[@id="filter_out_of_stock"]/div[2]/a/label/div/span').click()
    time.sleep(4)
    clickTipologia=driver.find_element_by_xpath('//*[@id="filter_hoteltype"]/div[2]/a[2]/label/div').click()
    
    time.sleep(4)
    accommodations_urls = list()
    accommodations_data = list()

    for accomodation_title in driver.find_elements_by_class_name('sr-hotel__title'):
        accommodations_urls.append(accomodation_title.find_element_by_class_name('hotel_name_link').get_attribute('href'))

    for url in range(0, n_results):   #il numero di controllo hotel nel try
        if url == n_results:
            break
        url_data = scrape_dati_alloggi(driver, accommodations_urls[url])
        accommodations_data.append(url_data)
    
    return accommodations_data

def scrape_dati_alloggi(driver, accommodation_url):  #visita le singole pagine degli hotel e inizia la raccolta dati

    if driver == None:
        driver = prepare_driver(accommodation_url)

    driver.get(accommodation_url)
    time.sleep(5)

    campi_alloggi = dict() #dizionario dove salvare i risultati
    
    campi_alloggi['data odierna']=dataOdierna
    campi_alloggi['citta']=stringaCitta  #nome della città
    campi_alloggi['disponibilita odierna']=numstruttureTrovate
    campi_alloggi['nome'] = driver.find_element_by_id('hp_hotel_name').text #restituisce il nome hotel
    campi_alloggi['tipologia struttura']=driver.find_element_by_xpath('//*[@id="hp_hotel_name"]/span').text
    campi_alloggi['prezzo'] = driver.find_element_by_class_name('hprt-table-cheapest-block').find_element_by_class_name('prco-valign-middle-helper').text.replace("\u20ac ","")
    
    return campi_alloggi

if __name__ == '__main__':

    try:
        for city in cities:    
            url = "https://www.booking.com/searchresults.it.html?checkin_month={in_month}&checkin_monthday={in_day}&checkin_year={in_year}&checkout_month={out_month}&checkout_monthday={out_day}&checkout_year={out_year}&group_adults={people}&group_children=0&order=distance_from_search&ss={city}%2C%20{country}".format(in_month=in_month, 
            in_day=in_day, #metodo per formattare e inserire le variabili nella url, ordinato per vicinanza dal centro e non per popularity o price
            in_year=in_year, 
            out_month=out_month,
            out_day=out_day,
            out_year=out_year,
            people=people,
            city=city,
            country=country)
            driver = preparaDriver(url)
            stringaCitta=city #variabile da assegnare alla lista dei campi_alloggi
            #stringaCitta=driver.find_element_by_class_name('sr_header--title').find_element_by_tag_name('h2').text.split(' ', 1)[0]
            #struttureTrovate = driver.find_element_by_class_name('sr_header--title').find_element_by_tag_name('h2').text
            xpathStruttura = driver.find_element_by_xpath('//*[@role="heading"]').text  #percorso del titolo che varia a seconda delle citta
            numstruttureTrovate=(re.findall('[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?',xpathStruttura)[0]) #prende il primo numero in lista relativo alle strutture
            accommodations_data = scrape_risultati(driver, 2) #quanti hotel prendo in considerazione per ogni città
            #accommodations_data = json.dumps(accommodations_data, indent=4)
            keys = accommodations_data[0].keys()
            with open('aquila.csv', 'a', newline='') as csvFile: #appende ai dati precedenti 
                dict_writer = csv.DictWriter(csvFile, keys) # imposto i nomi dei campi
                dict_writer.writerows(accommodations_data)
                driver.close()
    finally:
        driver.quit()